# CGAN (Conditional GAN)

cGAN(conditional GANs, cGANs)은 GAN 모델을 확장한 것이다. 
cGAN을 사용하면 특정 조건이나 속성을 지닌 이미지를 생성할 수 있는데 결과적으로 vanlia GAN보다 더 나은 이미지가 생성된다. 

이번 cGAN을 통해 할 것 : 자동으로 얼굴 노화를 수행하는 것.
- cGAN 논문 : https://arxiv.org/pdf/1411.1784.pdf
- 구현하려는 신경망에 대한 논문 (Face aging with cCAN)  : https://arxiv.org/pdf/1702.01983.pdf

cGAN에서 다룰 내용
- 얼굴 노화 처리용 GAN 소개
- 프로젝트 구성
- 데이터 준비
- cGAN을 케라스로 구현
- cGAN 훈련
- evaluation 과 hypterparameter조율
- 실용적인 얼굴 노화 어플리케이션

## 얼굴 노화 처리용 GAN 소개
지금까지 우리는 각 용도별로 서로 다른 GAN 신경망들을 구현해 보았다. cGAN은 vanila GAN이라는 개념을 확장한 것으로 generator 신경망의 출력을 제어할 수 있다. 얼굴 노화란 사람의 정체성을 바꾸는 일 없이 그 사람의 얼굴 나이를 바꾸는 일을 말한다. 대부분의 다른 모델들에서는 얼굴 표정이나 선글라스나 턱수염과 같은 얼굴 장식을 고려하지 않으므로 사람의 외모나 정체성에 관한 정보 중에 50% 정도를 잃게 된다. Age-cGAN은 이러한 특성(attributes)들을 모두 고려한다.

### cGAN 이해하기
cGAN은 일부 추가 정보 (extra information)을 조건으로 삼는 GAN 유형이다. 우리는 추가 정보 y를 추가 입력 계층 형태로 generator에 공급한다. vanila GAN에서는 생성된 이미지들의 범주를 전혀 제어하지 않는다. generator에 조건 y를 추가하면 우리는 클래스 레이블이나 정수 데이터와 같은 어떤 데이터 종류일 수 있는 y를 사용해 특정 범주(categories)에 맞는 이미지를 생성할 수 있다. vanila GAN은 하나의 범주만 사용할 수 있으므로 여러 범주에 맞춰 GAN들은 설계하기 무척 어렵다. 


그러나 cGAN을 사용하면 범주별로 조건이 다른 multi-modal models, 즉 다중 최빈값 모델을 생성할 수 있다.

먼저 기존 GAN의 공식을 알아보자.
<img src='img/gan.png'>

진짜 이미지와 비슷한 이미지를 생성하는 generator 신경망을 만들기 위해서는 우리는 generator가 생성된 데이터와 진짜 데이터 간의 유사도를 높여야 한다. 그러므로 object function을 사용하여 유사도를 측정한다. 


훈련 도중 D는 총 출력을 최대화하기를 바라지만 G는 최소화하기 바란다. 그래서 GAN을 훈련하게 되면 이 생성기 신경망과 판별기 신경망이 균형이 잡히게 된다. 여기서 GAN이 균형을 잡게 되었을 때 우리는 모델이 수렴했다고 한다. 이 균형은 내쉬 균형이다. (GAN에 대한 설명은 다른 장에서 설명)

그렇다면 cGAN을 알아보자.


<img src='img/cgan_img.png'>



GAN에 비해 cGAN이 다른점은 의외로 간단하다. G와 D에 특정 condition을 나타내는 정보 y를 가해주면 된다.

여기서 y의 형태는 특별하게 정해진 것이 아니기 때문에 다양한 형태를 가질 수 있다. 예를 들어, MNIST에서 원하는 숫자를 generator하고 싶다면 숫자의 class에 해당하는 label(one-hot-encoding)을 추가로 넣어주면 된다. 만약 MNIST를 one-hot-encoding을 하면 10bit가 필요하기 때문에 y는 10bit가 된다. 이뿐만 아니라 다양한 형태도 가능한 mutil modal condition 또한 가능하다.

아키텍처를 살펴보면 Age-cGAN은 네 개의 신경망으로 구성되는데 Encoder, FaceNet, Generator, Discriminator로 구성된다.

1. Encoder의 사용: 입력 이미지들의 역사상(inverse mapping)과 잠재적 벡터를 사용하는 나이 조건을 학습
2. FaceNet의 사용 : 얼굴 인식 신경망으로 입력 이미지 x개와 재구성한 이미지인 x'간의 차이를 학습
3. Generator의 사용 : 얼굴 이미지와 상태 벡터로 구성된 은닉 표현을 취하여 이미지를 생성
4. Discriminator의 사용 : 진짜 이미지와 가짜 이미지를 판별해 내기위한 것


cGAN의 문제점 → 속성 y가 있는 입력 이미지 x를 잠재 벡터 z에 inverse mapping하는 작업을 할 수 없다는 점이다. 이때 encoder 신경망을 사용하는 것이 이 문제를 해결하는 방법이다. 우리는 입력이미지에 해당하는 x의 inverse mapping을 근사하기 위해 encoder 신경망을 훈련할 수 있다. 위 4개의 신경망에 대해 좀더 자세히 알아보자.

### 1. Encoder
공급 받은 이미지들의 잠재 벡터를 생성하는 게 encoder 신경망의 첫 목표이다. encoder는 기본적으로 (64, 64, 3)차원으로 된 이미지 한 장을 취해 100차원의 벡터로 바꾼다. encoder는 심층 합성곱 신경망이다. 이 신경망에는 네 개의 conv block과 두 개의 dense layers로 구성된다. 각 conv block에는 conv layer 한 개, batchnorm layer 한 개, activation function 한 개를 담는다. 각 conv block에서 각 conv layer는 첫 번째 conv layer을 제외하고는 batchnorm layer가 따라 온다. 


### 2. Generator
G의 주요 목표는 (64, 64, 3) 차원으로 된 이미지를 생성하는 데 있다. G는 100차원으로 된 잠재 벡터와 일부 부가 정보인 y를 취해 진짜 같은 이미지들을 생성하려고 시도한다. G 또한 심층 합성곱 신경망이다. Dense layer과 상향 표본 추출 계층 및 conv layer으로 구성된다. 잡음 벡터와 conditioning value라는 두가지 값을 취한다. conditioning value는 신경망에 공급하는 부가 정보를 의미한다. Age-cGAN의 경우는 나이가 이에 해당한다. 


### 3. Discriminator
공급된 이미지가 진짜인지 아닌지 식별해내는 것이 D의 핵심 목표이다. D는 해당 이미지를 연달아 있는 하향 표본추출 계층들과 일부 분류 계층들을 거치게 전달하는 식으로 이미지를 식별해낸다. 다시 말해 D는 이미지가 진짜인지 아니면 가짜인지를 예측하는 것이다. 이 또한 D도 심층 합성곱 신경망이다. D에서는 여러 개의 conv block이 있다. 각 conv block에는 conv layer 한 개, batchnorm 한 개, activation function 한 개가 들어있지만 G처럼 첫 번째 conv block에는 예외적으로 batchnorm layer가 없다.


### 4. FaceNet
주어진 이미지를 바탕으로 특정인의 정체성(identity)을 인식하는 것이 face recognition의 핵심이다. 우리의 과정에서는 pre-trained Inception-ResNet-2 모델을 사용하고 이 모델에는 Fully connected layer가 없다. 이 모델에 일단 이미지 한개를 공급하면 이에 대응하는 임베딩을 돌려준다. 진짜 이미지와 재구성 이미지에서 추출한 임베딩을 계산할 때는 임베딩의 유클리드 거리를 계산한다. (임베딩 및 유클리드 거리는 다음장에 설명)

---

## Age-cGAN의 훈련 단계

1. cGAN 훈련 : G와 D를 훈련한다.
2. 초기 잠재 벡터 근사(Initial latent vector approximation) : encoder를 훈련한다.
3. 잠재 벡터 최적화(latent vector optimization) : encoder 및 generator를 모두 최적화 한다.

<img src='img/age-cgan.png'>
출처 : https://arxiv.org/pdf/1702.01983.pdf

### 1. cGAN 훈련
이 단계에서는 G와 D를 훈련한다. 훈련을 하고 나면 G는 흐릿하게 처리한 얼굴 이미지를 생성할 수 있다. 이 단계에서 vanila GAN 훈련 때와 비슷하게 훈련을 진행한다.

#### Object function
cGAN에서는 아래와 같은 object function을 사용할 수 있다.

<img src='img/cgan.png'>

cGAN 을 훈련하는 일은 함수 v(θG, θD)를 최적화하는 일과 관련이 있다. cGAN 훈련을 MinMax 게임으로 생각할 수 있다. 여기서 log D(x, y)는 D의 손실이고 [log (1 − D(G(z, ye), ye))는  G의 손실이며 Pdata는  도는 있음직한 이미지들의 분포이다.


### 2. 초기 잠재 벡터 근사 (Initial latent vector approximation)
초기 잠재 벡터 근사란 얼굴 이미지 재구성 작업을 최적화하기 위해 잠재 벡터에 근사하기 위한 방법이다. 잠재 벡터에 근사하기 위해 우리는 encoder 한 개를 사용한다. 우리는 생성된 이미지와 진짜 이미지를 가지고 encoder를 훈련한다. 훈련을 마치고 나면 encoder는 학습한 분포에서 잠재 벡터(latent vector)들을 생성해내기 시작한다. encoder 를 훈련하기 위한 object function 훈련은 유클리드 거리 손실(Euclidean distance loss) 에 해당한다.

### 3. 잠재 벡터 최적화 (latent vector optimization)
잠재 벡터 최적화를 하는 동안에 우리는 encoder와 generator를 동시에 최적화한다. 잠재 벡터 최적화를 위해 사용하는 방정식은 아래에 해상한다.

<img src='img/optimization.png'>
          
FR은 Face Recognition 신경망이다. 이 방정식은 진짜 이미지와 재구성한 이미지들간의 유클리드 거리를 가르키는데 이 유클리드 거리가 최소가 되어야 한다. 이번 단계에서 우리는 해당 거리를 최소가 되게 함으로써 정체성 보존을 최대화하려고 노력할 것이다.

### Reference
- http://blog.naver.com/PostView.nhn?blogId=laonple&logNo=221306150417
- 실전! GAN 프로젝트